In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df= pd.read_csv('../input/water-potability/water_potability.csv')
df.head()

In [ ]:
features_with_nan = [feature for feature in df.columns if df[feature].isnull().sum()>0]
features_with_nan

In [ ]:
df.dtypes

In [ ]:
df['Trihalomethanes'].isna().sum()

In [ ]:
for feature in df.columns:
    fig = plt.figure(figsize =(10, 7))
    df.boxplot(feature)
    plt.title(feature)
    plt.show()

In [ ]:
df['Sulfate'].describe()

In [ ]:
df.shape

In [ ]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
df['Potability'].value_counts()

In [ ]:
class_0 = df[df['Potability']==0]
class_1 = df[df['Potability']==1]
from sklearn.utils import resample
df_minority_resampled = resample(class_1,n_samples=1200,replace=True) 
df = pd.concat([class_0,df_minority_resampled])
df['Potability'].value_counts()

In [ ]:
X = df.drop('Potability', axis = 'columns')
y = df['Potability']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1,random_state=42)
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

clf_dt = DecisionTreeClassifier(random_state=42)
clf_dt = clf_dt.fit(X_train,y_train)

In [ ]:
plt.figure(figsize = (15,7.5))
plot_tree(clf_dt,
    max_depth=None,
    feature_names=None,
    class_names=['Not to drink','Can Drink'],
    label='all',
    filled=False,
    impurity=True,
    node_ids=False,
    proportion=False,
    rotate='deprecated',
    rounded=False,
    precision=3,
    ax=None,
    fontsize=None,)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,clf_dt.predict(X_test)))

In [ ]:
plot_confusion_matrix(clf_dt, X_test,y_test, display_labels=['not to drink','drinkable'])

In [ ]:
from sklearn.model_selection import cross_val_score
path = clf_dt.cost_complexity_pruning_path(X_train,y_train)
ccp_alphas = path.ccp_alphas
ccp_alphas = ccp_alphas[:-1]

alpha_loop_values = []

for ccp_alpha in ccp_alphas:
    clf_dt = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    scores = cross_val_score(clf_dt,X_train,y_train,cv=5)
    alpha_loop_values.append([ccp_alpha,np.mean(scores)])

In [ ]:
alpha_df = pd.DataFrame(alpha_loop_values,columns= ['ccp_alpha', 'MeanScores'])

alpha_df.plot('ccp_alpha','MeanScores', marker='o')
 

In [ ]:
alpha_df[alpha_df['MeanScores']>0.700]

In [ ]:
clf_dt = DecisionTreeClassifier(random_state=42, ccp_alpha=0)
clf_dt = clf_dt.fit(X_train,y_train)

In [ ]:
plot_confusion_matrix(clf_dt, X_test,y_test, display_labels=['not to drink','drinkable'])

In [ ]:
print(classification_report(y_test,clf_dt.predict(X_test)))

In [ ]:
clf_dt.score(X_test,y_test)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)
###new
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
clf_rf = GridSearchCV(RandomForestClassifier(),{
    'n_estimators' : [100,200,350,500,800,850],
    'min_samples_leaf' : [2,10,30]
}, cv=5)
clf_rf.fit(X_train_std,y_train)

In [ ]:
clf_rf.best_params_

In [ ]:
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier(min_samples_leaf=2,n_estimators=850)
rf.fit(X_train_std,y_train)
print(classification_report(y_test, rf.predict(X_test_std)))

In [ ]:
plot_confusion_matrix(rf, X_test_std,y_test, display_labels=['not to drink','drinkable'])